In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 # For loading the videofile and plotting bounding boxes
import os # For loading datafiles
from matplotlib import pyplot as plt # For plotting the annotations

# Input data files are available in the "../input/" directory.
# Loading data, the data is semicolon seperated.

df_valid  = pd.read_csv("dataset/annotations/annotations_AAU/valid.csv", sep=";")
df_train = pd.read_csv("dataset/annotations/annotations_AAU/train.csv", sep=";")
df_test = pd.read_csv("dataset/annotations/annotations_AAU/test.csv", sep=";")


In [4]:
df_valid

,Filename,Object ID,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y
0,2019-02-21_06-52-16to2019-02-21_06-52-34_1-003...,3,starfish,431,459,475,505
1,2019-03-06_22-11-06to2019-03-06_22-11-17_1-015...,33,crab,640,365,680,385
2,2019-03-06_22-11-06to2019-03-06_22-11-17_1-015...,34,crab,735,325,758,350
3,2019-03-06_22-11-06to2019-03-06_22-11-17_1-015...,35,crab,733,487,762,516
4,2019-03-06_22-11-06to2019-03-06_22-11-17_1-015...,141,crab,798,502,830,533
...,...,...,...,...,...,...,...
3576,2019-03-20_06-00-48to2019-03-20_06-00-56_1-009...,7,crab,374,311,418,338
3577,2019-02-26_00-44-33to2019-02-26_00-44-47_1-002...,17,starfish,371,413,397,479
3578,2019-02-26_00-44-33to2019-02-26_00-44-47_1-002...,108,starfish,252,449,301,490
3579,2019-03-19_11-42-56to2019-03-19_11-43-14_1-015...,24,fish,1,95,98,212


In [ ]:
from ultralytics import YOLO

def main():
    model = YOLO("yolov8s.pt")

    model.train(
        data="data/dataset.yaml",
        epochs=100,
        imgsz=960,
        batch=8,
        device=0,
        workers=8,
        patience=20,
        project="runs",
        name="marine_train"
    )

if __name__ == "__main__":
    main()

In [ ]:
from ultralytics import YOLO
import cv2
from pathlib import Path

VIDEO_PATH = "dataset/dataset/videos/jellyfish/2019-03-20_15-15-12to2019-03-20_15-15-19_1.avi"
MODEL_PATH = "weights/best.pt"  # или yolov8s.pt

OUTPUT_PATH = "output_detected.mp4"

def main():
    model = YOLO(MODEL_PATH)

    results = model.predict(
        source=VIDEO_PATH,
        conf=0.25,
        iou=0.5,
        save=True,
        project="runs",
        name="marine_detection",
        stream=False
    )

    print("Detection finished")


if __name__ == "__main__":
    main()